In [1]:
from glob import glob
from tqdm import tqdm
import json
import numpy as np
import os
from transformers import AutoTokenizer, WhisperConfig
from sklearn.feature_extraction.text import CountVectorizer

config = WhisperConfig.from_pretrained('openai/whisper-large-v3')
maxlen = config.max_length - 3

In [2]:
tokenizer = AutoTokenizer.from_pretrained('openai/whisper-large-v3')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
from datasets import Audio

sr = 16000
audio = Audio(sampling_rate=sr)

In [4]:
files = sorted(glob('output-imda/*.json'), key = lambda x: int(x.split('-')[-1].replace('.json', '')))
len(files)

68448

In [5]:
import re

pattern_pair = r'<\|(\d+\.\d+)\|>(.*?)<\|(\d+\.\d+)\|>'
matches = re.findall(pattern_pair, '<|0.00|> kerajaan persekutuan<|1.46|><|1.46|> dan banyak masalah hubungan<|3.96|><|3.96|> antara kerajaan negeri dan')
matches

[('0.00', ' kerajaan persekutuan', '1.46'),
 ('1.46', ' dan banyak masalah hubungan', '3.96')]

In [6]:
import string

punct = set(string.punctuation)

def remove_punct(s):
    return ''.join([c for c in s if c not in punct])

def remove_duplicate(string, n = 3):
    splitted = string.split()
    n = [splitted[i: i + n] for i in range(0, len(splitted), n)]
    already = set()
    dedup = []
    for n_ in n:
        original_n = ' '.join(n_)
        n_ = ' '.join(n_).lower()
        n_ = remove_punct(n_)
        if n_ not in already:
            dedup.append(original_n)
            already.add(n_)
    return ' '.join(dedup)

In [7]:
import math

def round_to_nearest_0_02(number):
    return round(number * 50) / 50

In [8]:
selected = [
    'terima kasih kerana menonton',
    'terima kasih',
]

In [9]:
with open(files[0]) as fopen:
    d = json.load(fopen)
    
d[0]

{'predict_ms': [50258,
  50282,
  50360,
  50365,
  430,
  1538,
  29216,
  7408,
  11,
  43365,
  3680,
  13877,
  992,
  3779,
  16281,
  717,
  289,
  657,
  282,
  12711,
  1706,
  66,
  12584,
  1706,
  73,
  9286,
  17289,
  74,
  892,
  449,
  7691,
  1988,
  23171,
  1026,
  40463,
  545,
  23059,
  569,
  5581,
  717,
  302,
  4579,
  3077,
  13,
  50715,
  50257,
  50257,
  50257,
  50257,
  50257,
  50257,
  50257,
  50257,
  50257],
 'predict_en': [50258,
  50259,
  50360,
  50365,
  4928,
  4104,
  82,
  365,
  3779,
  6352,
  645,
  14658,
  281,
  853,
  5145,
  641,
  1281,
  12126,
  2507,
  613,
  21688,
  4358,
  13,
  50715,
  50257,
  50257,
  50257,
  50257,
  50257,
  50257,
  50257,
  50257,
  50257],
 'score_ms': 10.375,
 'score_en': 8.75,
 'filename': 'IMDA-STT/part1-mp3/000010101.mp3'}

In [10]:
from huggingface_hub import hf_hub_download

In [11]:
f = hf_hub_download(
    repo_id="mesolitica/pseudolabel-imda-large-v3-timestamp", 
    filename="translate-imda-ms.json",
    repo_type = 'dataset'
)

In [12]:
with open(f) as fopen:
    translation = json.load(fopen)

len(translation)

1444550

In [13]:
keys = translation.keys()

In [14]:
popped = []
for k in list(keys):
    v = translation[k]
    v = re.sub(r'\s-', '-', v)
        
    translation[k] = v
    k_split = k.split('|')
    v_split = v.split('|')
    
    if len(k_split) != len(v_split):
        translation.pop(k, None)
        popped.append((k_split, v_split, k, v))

In [15]:
import mp
import copy

minimum_score = 5

def loop(files):
    
    tokenizer = AutoTokenizer.from_pretrained('openai/whisper-large-v3')
    
    files, _ = files
    results = []
    for f in tqdm(files):
        try:
            with open(f) as fopen:
                data = json.load(fopen)
        except:
            continue
        f_split = os.path.split(f)[-1].replace('.json', '')
        for i in range(len(data)):
            
            audio_filename = data[i]['filename']
            if not os.path.exists(audio_filename):
                continue
                
            y = audio.decode_example(audio.encode_example(audio_filename))['array']
            len_y = len(y) / sr
            if len_y > 30:
                continue
            rounded_num = f'<|{round_to_nearest_0_02(len_y):.2f}|>'
                    
            
            if data[i]['score_en'] > minimum_score:
                
                a = tokenizer._decode_asr(
                [{'tokens': np.array([data[i]['predict_en']])}], 
                return_timestamps = True, return_language = 'en', 
                time_precision = 0.02)[1]['chunks']
            
                a = [a_['text'] for a_ in a]
                
                t = ' <>'.join(a)
                if t not in translation:
                    continue
                    
                t_translated = translation[t]
                t_translated_splitted = t_translated.split('<>')
                if len(t_translated_splitted) != len(a):
                    continue
                
                mapping = {}
                for k in range(len(a)):
                    s = t_translated_splitted[k].strip()
                    if a[k][0] == ' ':
                        s = ' ' + s
                    if a[k][-1] == ' ':
                        s = s + ' '
                    mapping[a[k]] = s
                    
                a = np.array(data[i]['predict_en'])
                a = a[a != 50257].tolist() + [50257]
                t = tokenizer.decode(a, skip_special_tokens = True, decode_with_timestamps = True).strip()
                if t.split('|>')[-1] != '':
                    t += rounded_num
                
                matches = re.findall(pattern_pair, t)
                rs = []
                for match in matches:
                    l = float(match[0])
                    r = float(match[2])
                    t_ = match[1]
                    rt_ = re.sub('[^a-z ]+', '', t_.lower()).strip()
                    
                    if (r - l > 3) and any([s == rt_ for s in selected]):
                        # print(audio_filename, t_)
                        t_ = ''
                    else:
                        if t_ in mapping:
                            t_ = mapping[t_]
                        else:
                            t_ = ''
                    
                    splitted = t_.split()
                    if len(splitted):
                        ratio = (len(set(splitted)) / len(splitted))
                        if len(t_) > 100 and ratio < 0.5:
                            t_ = remove_duplicate(t_)
                    if len(t_) and t_[0] != ' ':
                        t_ = ' ' + t_
                    rs.append(f'<|{match[0]}|>{t_}<|{match[2]}|>')
                rs = ''.join(rs)
                t = f'<|startoftranscript|><|ms|><|transcribe|>{rs}<|endoftext|>'
                d = {
                    'new_text': t,
                    'audio_filename': audio_filename,
                }
                results.append(d)
    return results

In [16]:
results = loop((files[:100], 0))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
100%|██████████| 100/100 [00:24<00:00,  4.09it/s]


In [17]:
len(results)

3316

In [18]:
results[-10]['new_text']

'<|startoftranscript|><|ms|><|transcribe|><|0.00|> Ini adalah kuota kedua di mana kadar pertumbuhan kenderaan sifar telah digunakan.<|5.12|><|endoftext|>'

In [19]:
results[0]['new_text']

'<|startoftranscript|><|ms|><|transcribe|><|0.00|> Isi rumah dengan set sasaran digalakkan untuk cuba mengekalkan penggunaan air mereka di bawah paras yang ditetapkan ini.<|7.00|><|endoftext|>'

In [20]:
tokenizer = AutoTokenizer.from_pretrained('openai/whisper-medium')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [21]:
tokenizer.tokenize(results[-1]['new_text'])

['<|startoftranscript|>',
 '<|ms|>',
 '<|transcribe|>',
 '<|0.00|>',
 'ĠKorea',
 'ĠUt',
 'ara',
 'Ġm',
 'esti',
 'Ġmend',
 'apat',
 'kan',
 'Ġj',
 'alan',
 'Ġpul',
 'ang',
 'Ġke',
 'Ġme',
 'ja',
 '.',
 '<|3.38|>',
 '<|endoftext|>']

In [22]:
tokenizer.decode(tokenizer(results[-1]['new_text'])['input_ids'], decode_with_timestamps = True)

'<|startoftranscript|><|notimestamps|><|startoftranscript|><|ms|><|transcribe|><|0.00|> Korea Utara mesti mendapatkan jalan pulang ke meja.<|3.38|><|endoftext|><|endoftext|>'

In [23]:
[r for r in results if r['new_text'].count('<|') > 6]

[]

In [24]:
results = mp.multiprocessing(files, loop, cores = 100)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [25]:
len(results)

1217490

In [26]:
results[0]

{'new_text': '<|startoftranscript|><|ms|><|transcribe|><|0.00|> Isi rumah dengan set sasaran digalakkan untuk cuba mengekalkan penggunaan air mereka di bawah paras yang ditetapkan ini.<|7.00|><|endoftext|>',
 'audio_filename': 'IMDA-STT/part1-mp3/000010101.mp3'}

In [27]:
import IPython.display as ipd
ipd.Audio(results[0]['audio_filename'])

In [28]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(results, test_size = 300)

In [29]:
with open('prepared-imda-ms.jsonl', 'w') as fopen:
    for r in tqdm(train):
        fopen.write(f'{json.dumps(r)}\n')

100%|██████████| 1217190/1217190 [00:06<00:00, 202195.51it/s]


In [30]:
!ls -lh prepared-imda-ms.jsonl

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


-rw-r--r-- 1 ubuntu ubuntu 251M Apr 26 11:18 prepared-imda-ms.jsonl


In [31]:
from huggingface_hub import HfApi
api = HfApi()

In [32]:
api.upload_file(
    path_or_fileobj='prepared-imda-ms.jsonl',
    path_in_repo='prepared-imda-ms.jsonl',
    repo_id='mesolitica/pseudolabel-imda-large-v3-timestamp',
    repo_type='dataset',
)

prepared-imda-ms.jsonl:   0%|          | 0.00/263M [00:00<?, ?B/s]

'https://huggingface.co/datasets/mesolitica/pseudolabel-imda-large-v3-timestamp/blob/main/prepared-imda-ms.jsonl'

In [33]:
with open('test-imda-dataset-ms.json', 'w') as fopen:
    json.dump(test, fopen)

In [34]:
api.upload_file(
    path_or_fileobj='test-imda-dataset-ms.json',
    path_in_repo='manglish-ms/test-imda-dataset-ms.json',
    repo_id='mesolitica/speech-test-set',
    repo_type='dataset',
)

test-imda-dataset-ms.json:   0%|          | 0.00/63.6k [00:00<?, ?B/s]

'https://huggingface.co/datasets/mesolitica/speech-test-set/blob/main/manglish-ms/test-imda-dataset-ms.json'

In [35]:
!rm -rf manglish-ms
!mkdir manglish-ms

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [36]:
import shutil

for t in test:
    f = os.path.split(t['audio_filename'])[1]
    shutil.copyfile(t['audio_filename'], os.path.join('manglish-ms', f)) 
    
len(glob('manglish-ms/*'))

300

In [38]:
# !sudo apt install zip -y

In [40]:
# !zip -r test-manglish-ms-audio.zip manglish-ms

In [41]:
api.upload_file(
    path_or_fileobj='test-manglish-ms-audio.zip',
    path_in_repo='manglish-ms/test-manglish-ms-audio.zip',
    repo_id='mesolitica/speech-test-set',
    repo_type='dataset',
)

test-manglish-ms-audio.zip:   0%|          | 0.00/8.95M [00:00<?, ?B/s]

'https://huggingface.co/datasets/mesolitica/speech-test-set/blob/main/manglish-ms/test-manglish-ms-audio.zip'